In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

C:\Users\HARSHIT JAIN\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv(r'C:\Users\HARSHIT JAIN\Desktop\data science projects\Real_Estate_Data_science_Project\DATASETS\gurgaon_properties_post_feature_selection_v4\gurgaon_properties_post_feature_selection_v4.csv')

In [3]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,falt,sector 54,5.75,3,4,3,Relatively New,2329.0,1,0,furnished,high,high floor
1,falt,sector 2,3.35,4,4,3+,New Property,5000.0,0,0,semifurnished,medium,low floor
2,falt,sector 69,1.40,3,3,2,Relatively New,1428.0,0,0,semifurnished,medium,medium floor
3,falt,sector 79,1.70,3,2,3,Relatively New,1570.0,0,0,semifurnished,low,medium floor
4,falt,sector 33,1.35,3,2,3+,New Property,1409.0,0,0,semifurnished,low,high floor


In [4]:
df = df.replace({'property_type':{'falt':'flat'}})

In [5]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 54,5.75,3,4,3,Relatively New,2329.0,1,0,furnished,high,high floor
1,flat,sector 2,3.35,4,4,3+,New Property,5000.0,0,0,semifurnished,medium,low floor
2,flat,sector 69,1.40,3,3,2,Relatively New,1428.0,0,0,semifurnished,medium,medium floor
3,flat,sector 79,1.70,3,2,3,Relatively New,1570.0,0,0,semifurnished,low,medium floor
4,flat,sector 33,1.35,3,2,3+,New Property,1409.0,0,0,semifurnished,low,high floor


In [8]:
df.drop(columns=['store room','floor_category','balcony'],inplace=True)

In [9]:
df

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,flat,sector 54,5.75,3,4,Relatively New,2329.0,1,furnished,high
1,flat,sector 2,3.35,4,4,New Property,5000.0,0,semifurnished,medium
2,flat,sector 69,1.40,3,3,Relatively New,1428.0,0,semifurnished,medium
3,flat,sector 79,1.70,3,2,Relatively New,1570.0,0,semifurnished,low
4,flat,sector 33,1.35,3,2,New Property,1409.0,0,semifurnished,low
...,...,...,...,...,...,...,...,...,...,...
3550,flat,sector 83,1.42,3,3,Relatively New,1972.0,0,unfurnished,medium
3551,flat,sector 56,0.65,1,1,Moderately Old,839.0,0,semifurnished,medium
3552,house,sector 6,0.40,4,2,Moderately Old,1600.0,0,semifurnished,low
3553,flat,sector 89,1.28,3,3,New Property,1611.0,0,semifurnished,medium


In [10]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished

In [11]:
# Numerical = bedRoom, bathroom, built_up_area, servant room
# Ordinal = property_type, furnishing_type, luxury_category 
# OHE = sector, agePossession

In [12]:
df['agePossession'] = df['agePossession'].replace(
    {
        'Relatively New':'new',
        'Moderately Old':'old',
        'New Property' : 'new',
        'Old Property' : 'old',
        'Under Construction' : 'under construction'
    }
)

In [13]:
df.head()

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,flat,sector 54,5.75,3,4,new,2329.0,1,furnished,high
1,flat,sector 2,3.35,4,4,new,5000.0,0,semifurnished,medium
2,flat,sector 69,1.40,3,3,new,1428.0,0,semifurnished,medium
3,flat,sector 79,1.70,3,2,new,1570.0,0,semifurnished,low
4,flat,sector 33,1.35,3,2,new,1409.0,0,semifurnished,low


In [14]:
df['property_type'] = df['property_type'].replace({'flat':0,'house':1})

C:\Users\HARSHIT JAIN\AppData\Local\Temp\ipykernel_16636\71934247.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({'flat':0,'house':1})


In [15]:
df.head()

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,0,sector 54,5.75,3,4,new,2329.0,1,furnished,high
1,0,sector 2,3.35,4,4,new,5000.0,0,semifurnished,medium
2,0,sector 69,1.40,3,3,new,1428.0,0,semifurnished,medium
3,0,sector 79,1.70,3,2,new,1570.0,0,semifurnished,low
4,0,sector 33,1.35,3,2,new,1409.0,0,semifurnished,low


In [18]:
df['luxury_category'] = df['luxury_category'].replace({'low':0,'medium':1,'high':2})

C:\Users\HARSHIT JAIN\AppData\Local\Temp\ipykernel_16636\1896554504.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['luxury_category'] = df['luxury_category'].replace({'low':0,'medium':1,'high':2})


In [19]:
df.head()

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,0,sector 54,5.75,3,4,new,2329.0,1,furnished,2
1,0,sector 2,3.35,4,4,new,5000.0,0,semifurnished,1
2,0,sector 69,1.40,3,3,new,1428.0,0,semifurnished,1
3,0,sector 79,1.70,3,2,new,1570.0,0,semifurnished,0
4,0,sector 33,1.35,3,2,new,1409.0,0,semifurnished,0


In [23]:
df['furnishing_type'].unique()

array(['furnished', 'semifurnished', 'unfurnished'], dtype=object)

In [24]:
df['furnishing_type'] = df['furnishing_type'].replace({'furnished':2,'semifurnished':1,'unfurnished':0})

C:\Users\HARSHIT JAIN\AppData\Local\Temp\ipykernel_16636\3383776019.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['furnishing_type'] = df['furnishing_type'].replace({'furnished':2,'semifurnished':1,'unfurnished':0})


In [25]:
df.head()

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,0,sector 54,5.75,3,4,new,2329.0,1,2,2
1,0,sector 2,3.35,4,4,new,5000.0,0,1,1
2,0,sector 69,1.40,3,3,new,1428.0,0,1,1
3,0,sector 79,1.70,3,2,new,1570.0,0,1,0
4,0,sector 33,1.35,3,2,new,1409.0,0,1,0


In [30]:
 pd.get_dummies(df,columns=['sector','agePossession'],drop_first=True)

,property_type,price,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,...,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sector 99a,sector_sector 9a,sector_sohna road,sector_sohna road road,agePossession_old,agePossession_under construction
0,0,5.75,3,4,2329.0,1,2,2,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,3.35,4,4,5000.0,0,1,1,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,1.40,3,3,1428.0,0,1,1,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,1.70,3,2,1570.0,0,1,0,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,1.35,3,2,1409.0,0,1,0,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550,0,1.42,3,3,1972.0,0,0,1,False,False,...,False,False,False,False,False,False,False,False,False,False
3551,0,0.65,1,1,839.0,0,1,1,False,False,...,False,False,False,False,False,False,False,False,True,False
3552,1,0.40,4,2,1600.0,0,1,0,False,False,...,False,False,False,False,False,False,False,False,True,False
3553,0,1.28,3,3,1611.0,0,1,1,False,False,...,False,False,False,False,False,False,False,False,False,False


In [35]:
new_df = pd.get_dummies(df,columns=['sector','agePossession'],drop_first=True).astype('int')

In [36]:
new_df

,property_type,price,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,...,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sector 99a,sector_sector 9a,sector_sohna road,sector_sohna road road,agePossession_old,agePossession_under construction
0,0,5,3,4,2329,1,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,4,4,5000,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,3,3,1428,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,3,2,1570,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,3,2,1409,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550,0,1,3,3,1972,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3551,0,0,1,1,839,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3552,1,0,4,2,1600,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3553,0,1,3,3,1611,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
new_df.shape

(3555, 124)

In [38]:
X = new_df.drop(columns=['price'])
y = new_df['price']

In [39]:
X

,property_type,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,sector_new,...,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sector 99a,sector_sector 9a,sector_sohna road,sector_sohna road road,agePossession_old,agePossession_under construction
0,0,3,4,2329,1,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,4,4,5000,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,3,1428,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,3,2,1570,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,3,2,1409,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550,0,3,3,1972,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3551,0,1,1,839,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3552,1,4,2,1600,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3553,0,3,3,1611,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
y

0       5
1       3
2       1
3       1
4       1
       ..
3550    1
3551    0
3552    0
3553    1
3554    0
Name: price, Length: 3555, dtype: int32

In [41]:
y_log = np.log1p(y)

In [42]:
y_log

0       1.791759
1       1.386294
2       0.693147
3       0.693147
4       0.693147
          ...   
3550    0.693147
3551    0.000000
3552    0.000000
3553    0.693147
3554    0.000000
Name: price, Length: 3555, dtype: float64

In [43]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled,columns=X.columns)

In [44]:
X_scaled

,property_type,bedRoom,bathroom,built_up_area,servant room,furnishing_type,luxury_category,sector_gwal pahari,sector_manesar,sector_new,...,sector_sector 92,sector_sector 93,sector_sector 95,sector_sector 99,sector_sector 99a,sector_sector 9a,sector_sohna road,sector_sohna road road,agePossession_old,agePossession_under construction
0,-0.517088,-0.073838,0.505645,0.384555,1.337249,2.310808,1.867758,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901
1,-0.517088,0.728896,0.505645,2.580315,-0.747804,0.430102,0.441667,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901
2,-0.517088,-0.073838,-0.184136,-0.356134,-0.747804,0.430102,0.441667,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901
3,-0.517088,-0.073838,-0.873916,-0.239400,-0.747804,0.430102,-0.984424,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901
4,-0.517088,-0.073838,-0.873916,-0.371754,-0.747804,0.430102,-0.984424,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550,-0.517088,-0.073838,-0.184136,0.091074,-0.747804,-1.450604,0.441667,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901
3551,-0.517088,-1.679305,-1.563697,-0.840336,-0.747804,0.430102,0.441667,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,1.647645,-0.190901
3552,1.933908,0.728896,-0.873916,-0.214738,-0.747804,0.430102,-0.984424,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,1.647645,-0.190901
3553,-0.517088,-0.073838,-0.184136,-0.205695,-0.747804,0.430102,0.441667,-0.071338,-0.093791,-0.033563,...,-0.170128,-0.050379,-0.125357,-0.058198,-0.092253,-0.053112,-0.211344,-0.055712,-0.606927,-0.190901


In [45]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(LinearRegression(), X_scaled, y_log, cv=kfold, scoring='r2')

In [46]:
scores.mean(),scores.std()

(0.8041491983766285, 0.022153863455282016)

In [48]:
lr = LinearRegression()

In [49]:
lr.fit(X_scaled,y_log)

LinearRegression()

In [50]:
lr.coef_

array([ 1.27742901e-01,  7.69021187e-02,  7.94311213e-02,  2.42596728e-01,
        6.66885284e-02, -3.72861469e-03,  4.60739389e-03,  5.19257686e-03,
       -5.08715262e-02, -8.87006186e-03, -3.00689665e-03, -1.63292231e-02,
        1.48011490e-02, -1.19320798e-02, -1.24807515e-02, -2.87526296e-02,
       -5.58003845e-03, -3.04049469e-02,  1.57509778e-03,  1.44703570e-02,
        3.61478610e-04, -2.52100362e-02,  7.76449459e-03,  9.73234138e-03,
        2.80712573e-02,  2.99919794e-02, -3.15018126e-02, -2.00140851e-02,
        2.95707936e-02,  1.36368124e-03,  6.09080164e-03,  3.79555807e-03,
        1.22574273e-02, -7.57730241e-03, -1.71044139e-02,  3.46805155e-02,
       -7.27410188e-04,  1.42325394e-02,  5.38728490e-02,  7.26213401e-02,
        6.34611242e-03,  4.12811453e-02, -2.72939704e-02, -1.36327737e-02,
       -2.37972487e-02,  1.52005389e-03,  2.25088872e-02,  2.78129041e-02,
       -1.98097340e-02,  6.96317827e-03,  1.18124090e-02, -3.23382029e-02,
       -7.54202552e-03,  

In [52]:
lr.coef_.shape

(123,)

In [55]:
coef_df =  pd.DataFrame(lr.coef_.reshape(1,123),columns=X.columns).stack().reset_index().drop(columns=['level_0']).rename(columns={'level_1':'feature',0:'coef'})

In [56]:
import statsmodels.api as sm

X_with_const = sm.add_constant(X_scaled)

model = sm.OLS(y_log, X_with_const).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     128.5
Date:                Wed, 18 Sep 2024   Prob (F-statistic):               0.00
Time:                        12:11:05   Log-Likelihood:                -637.15
No. Observations:                3555   AIC:                             1522.
Df Residuals:                    3431   BIC:                             2288.
Df Model:                         123                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 